# Bedrock AgentCore Gateway を使用して Smithy API を MCP ツールに変換する

## 概要
Bedrock AgentCore Gateway は、お客様が既存の Smithy API をインフラやホスティングを管理することなく、フルマネージドの MCP サーバーに変換する方法を提供します。お客様は Smithy スペックを持ち込み、MCP ツールに変換できます。ここでは、Amazon S3 の Smithy モデルから MCP ツールを作成するデモを行います。エージェントは Amazon S3 にクエリを実行し、関連する質問に回答できるようになります。

Gateway のワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます：
* **Gateway 用のツールを作成** - Smithy 仕様を使用してツールを定義します。
* **Gateway エンドポイントを作成** - インバウンド認証を備えた MCP エントリポイントとして機能するゲートウェイを作成します。
* **Gateway にターゲットを追加** - ゲートウェイがリクエストを特定のツールにルーティングする方法を定義する Smithy ターゲットを設定します。Smithy ファイルに含まれるすべての操作が MCP 互換ツールとなり、Gateway エンドポイント URL を通じて利用可能になります。Smithy 経由で Amazon S3 API を呼び出すための AWS IAM を使用したアウトバウンド認可を設定します。
* **エージェントコードを更新** - 統合された MCP インターフェースを通じてすべての設定済みツールにアクセスするため、エージェントを Gateway エンドポイントに接続します。

![仕組み](images/smithy-apis-gateway.png)

### チュートリアル詳細

| 情報                  | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ    | インタラクティブ                                           |
| AgentCore コンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agents                                            |
| Gateway ターゲットタイプ | Smithy                                                   |
| エージェント           | インフラストラクチャエージェント                            |
| インバウンド認証 IdP    | Amazon Cognito                                            |
| アウトバウンド認証      | IAM                                                       |
| LLM モデル             | Anthropic Claude Haiku 4.5, Amazon Nova Pro              |
| チュートリアル構成      | AgentCore Gateway の作成と呼び出し                         |
| チュートリアル業界      | クロスバーティカル                                         |
| 難易度                 | 簡単                                                       |
| 使用 SDK               | boto3                                                     |

チュートリアルの最初のパートでは、AgentCore Gateway ターゲットを作成します。

### チュートリアルアーキテクチャ
このチュートリアルでは、Amazon S3 の Smithy スペックで定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。これにより、ユーザーは自身の AWS アカウント内の Amazon S3 に関するあらゆる質問をすることができます。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Python 3.10+）
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import os
# Set AWS credentials if not using SageMaker notebook
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role_s3_smithy("sample-lambdagateway")
print("Agentcore Gateway ロール ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認可用に Amazon Cognito プールを作成

In [ ]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognito リソースを作成または取得中...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"ユーザープール ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("リソースサーバーを確保しました。")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"クライアント ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Gateway を作成

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoS3Smithyv3',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with Smithy target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Amazon S3 スペックで Smithy ターゲットを作成

Amazon S3 API の Smithy ゲートウェイターゲットを作成します。Amazon S3 の Smithy JSON ファイルをインポートし、API を MCP ツールに変換します。

#### S3 Smithy API json ファイルを S3 にアップロード

In [ ]:
# Create an S3 client
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# Retrieve AWS account ID and region
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters
bucket_name = f'agentcore-gateway-{account_id}-{region}' # Your s3 bucket to upload the OpenAPI json file.
file_path = 'smithy-specs/s3-apis.json'
object_key = 's3-apis.json'
# Upload the file using put_object and read response
try:
    try:
        if region == "us-east-1":
            s3bucket = s3_client.create_bucket(
                Bucket=bucket_name
            )
        else:
            s3bucket = s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={
                    'LocationConstraint': region
                }
            )
    except Exception as e:
        print(e)
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # Construct the ARN of the uploaded object with account ID and region
    smithy_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'アップロードされたオブジェクトの S3 URI: {smithy_s3_uri}')
except Exception as e:
    print(f'ファイルアップロードエラー: {e}')

#### Gateway ターゲットを作成

In [ ]:
# S3 Uri for Smithy spec file
smithy_s3_target_config = {
    "mcp": {
          "smithyModel": {
              "s3": {
                  "uri": smithy_s3_uri
              }
          }
      }
}

# IAM credentials provider configuration
credential_config  = {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }

targetname='DemoSmithytargetForS3'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Smithy Target with S3Uri using SDK',
    targetConfiguration=smithy_s3_target_config,
    credentialProviderConfigurations=[credential_config])

# Printing the request ID and timestamp for you to report the defects. Please include them while reporting issues/defects  
response_metadata = response['ResponseMetadata']
print(response_metadata)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装した Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合します。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

Bedrock AgentCore Gateway は、基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠のゲートウェイとして機能します。これらの API により、任意の MCP 準拠クライアントまたは SDK が利用可能なツールを発見し、安全で標準化された方法で対話できます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は [MCP 認可仕様](https://modelcontextprotocol.org/specification/draft/basic/authorization) に厳密に準拠しており、堅牢なセキュリティとアクセス制御を保証します。これは、Strands エージェントによるすべてのツール呼び出しが認可ステップを経ることを意味し、強力な機能を有効にしながらセキュリティを維持します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。Gateway はすべての必要なセキュリティ検証、プロトコル変換、サービス対話を処理し、プロセス全体をシームレスかつ安全にします。

このアーキテクチャアプローチにより、MCP 仕様を実装する任意のクライアントまたは SDK が Gateway を通じて AWS サービスと対話できるため、AI エージェント統合のための汎用性が高く将来を見据えたソリューションとなります。

# インバウンド認可用に Amazon Cognito からアクセストークンをリクエスト

In [ ]:
print("Amazon Cognito オーソライザーからアクセストークンをリクエスト中...ドメイン名の伝播が完了するまで失敗する場合があります")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("トークンレスポンス:", token)

# AWS アカウント内の S3 リソースについて IT エージェントに質問する

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    #print(f"エージェントに読み込まれたツール: {agent.tool_names}")
    #print(f"エージェントのツール設定: {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("すべての S3 バケットをリストしてもらえますか。適切なツールを見つけて、答えを得るためにツールを呼び出してください")
    #agent("火星の北部の天気はどうですか")
    # Invoke the agent with sample prompt, invoke the tool and display the response
    #agent("注文 ID 123 の注文状況を確認し、ツールからの正確なレスポンスを表示してください")
    #Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # You can replace this with unique identifier. 
    name=targetname+"___ListBuckets", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    #arguments={"ver": "1.0","feedtype": "json"}
    )
    #Print the MCP Tool response
    print(f"ツール呼び出し結果: {result['content'][0]['text']}")

# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されます。クリーンアップの一環として手動で削除する必要がある場合があります。これは実行する例によって異なります。

## Gateway を削除（オプション）

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)